In [1]:
import os, dotenv, json, uuid
dotenv.load_dotenv()

False

# 📥 Carga de datos

Cargue los datos de contexto en formato **JSON**.  
La estructura debe ser similar a la siguiente de referencia:

```json
{
    "Employees": ["E0", "E1"],
    "Desks": ["D0", "D1"],
    "Days": ["L", "Ma", "Mi", "J", "V"],
    "Groups": ["G0", "G1"],
    "Zones": ["Z0", "Z1"],
    "Desks_Z": {
        "Z0": ["D0"],
        "Z1": ["D1"]
    },
    "Desks_E": {
        "E0": ["D0", "D1"],
        "E1": ["D1"]
    },
    "Employees_G": {
        "G0": ["E0"],
        "G1": ["E1"]
    },
    "Days_E": {
        "E0": ["Mi"],
        "E1": ["L", "J"]
    }
}
```

Cargue su archivo en el siguiente campo antes de ejecutar el código:


In [2]:
# %%
import os
import json
import base64
import tempfile
import firebase_admin
from firebase_admin import credentials

def setup_firebase_credentials():
    """
    Setup Firebase credentials from environment variable and initialize Firebase app
    """
    # Check if Firebase app is already initialized
    try:
        return firebase_admin.get_app()
    except ValueError:
        # Get the base64 encoded credentials from environment variable
        firebase_credentials_base64 = os.getenv('GOOGLE_APPLICATION_CREDENTIALS_BASE64')
        
        if firebase_credentials_base64:
            # Decode the base64 credentials
            credentials_json = base64.b64decode(firebase_credentials_base64).decode('utf-8')
            
            # Create a temporary file to store the credentials
            with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as temp_file:
                json.dump(json.loads(credentials_json), temp_file)
                temp_file_path = temp_file.name
            
            # Set the environment variable to point to the temporary file
            os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = temp_file_path
            
            try:
                # Initialize Firebase app with the credentials
                cred = credentials.Certificate(temp_file_path)
                return firebase_admin.initialize_app(cred)
            finally:
                # Clean up the temporary file
                if os.path.exists(temp_file_path):
                    os.unlink(temp_file_path)
        return None


In [3]:
# %% Storage utilities for Deepnote notebook
import uuid
from dataclasses import dataclass
import json
from typing import Dict, Any

@dataclass
class FileRecord:
    content: bytes
    job_id: str | None = None

class Storage:
    def __init__(self):
        # In-memory mapping: file_id -> FileRecord
        self.FILES: Dict[uuid.UUID, FileRecord] = {}

    def save_file(self, content: bytes) -> uuid.UUID:
        """Save raw file content and return a unique file_id."""
        file_id = uuid.uuid4()
        self.FILES[file_id] = FileRecord(content=content)
        return file_id

    def load_json(self, file_id: uuid.UUID) -> Dict[str, Any]:
        """Load and parse JSON data from a previously saved file."""
        record = self.FILES.get(file_id)
        if not record:
            raise KeyError(f"No file found for file_id: {file_id}")
        return json.loads(record.content.decode())


In [4]:
from __future__ import annotations

import uuid
import threading
import logging
from dataclasses import dataclass
from typing import Dict, Any

from firebase_admin import firestore

# utilidades locales definidas en otras celdas
# (storage, run_optimization y setup_firebase_credentials)

# Initialize Firestore client
firebase_app = setup_firebase_credentials()
db = None
if firebase_app:
    db = firestore.client(firebase_app)
else:
    logging.warning("Firebase initialization failed. Some features may be unavailable.")

@dataclass
class JobRecord:
    file_id: uuid.UUID
    min_days: int
    status: str = "queued"
    result: Dict[str, Any] | None = None
    error: str | None = None
    log: str = ""

JOBS: Dict[str, JobRecord] = {}
storage = Storage()


def _run(job_id: str, file_id: uuid.UUID, min_days: int) -> None:
    job = JOBS[job_id]
    job.status = "running"
    try:
        data = storage.load_json(file_id)
        result = run_optimization(data, min_days)
        job.result = result
        job.status = "success"
        if db:
            # Convert data types for Firebase compatibility
            doc = {
                "status": "success",
                "assignments": result["assignments"],
                "group_zones_table": result["group_zones_table"],
                "kpis": result["kpis"],
                "timestamp": firestore.SERVER_TIMESTAMP,
                "minAttendance": min_days,
                "file_id": str(file_id),
                # New metrics
                "employees_g": {str(k): str(v) for k, v in result["employees_g"].items()},
                "meeting_days": {str(k): str(v) for k, v in result["meeting_days"].items()},
                "different_desks": result["different_desks"],
                "lonely_members": [
                    {"group": str(g), "employee": str(e), "day": str(d), "zone": str(z)}
                    for g, e, d, z in result["lonely_members"]
                ],
                "violated_preferences": [
                    {"employee": str(e), "day": str(d), "preferences": str(p) if not isinstance(p, (list, tuple)) else ', '.join(map(str, p))}
                    for e, d, p in result["violated_preferences"]
                ],
                "unused_desks": [
                    {"desk": str(desk), "day": str(day), "obs": str(obs)}
                    for desk, day, obs in result["unused_desks"]
                ]
            }
            db.collection("optimizations").document(job_id).set(doc)
    except Exception as exc:
        job.status = "failure"
        job.error = str(exc)
        if db:
            doc = {
                "status": "failure",
                "reason": str(exc),
                "timestamp": firestore.SERVER_TIMESTAMP,
                "minAttendance": min_days,
                "file_id": str(file_id),
            }
            db.collection("optimizations").document(job_id).set(doc)


def launch_job(file_id: uuid.UUID, min_days: int) -> str:
    job_id = str(uuid.uuid4())
    JOBS[job_id] = JobRecord(file_id=file_id, min_days=min_days)
    thread = threading.Thread(target=_run, args=(job_id, file_id, min_days), daemon=True)
    thread.start()
    record = storage.FILES[file_id]
    record.job_id = job_id
    storage.FILES[file_id] = record
    return job_id

def get_status(job_id: str) -> JobRecord:
    return JOBS[job_id]

In [ ]:
from __future__ import annotations

import time
from typing import Dict, Any, Tuple, List, Set
from pulp import (
    LpProblem,
    LpMaximize,
    LpVariable,
    LpBinary,
    LpInteger,
    lpSum,
    value,
    PULP_CBC_CMD,
    LpStatus,
)

def calculate_employees_by_group(Employees_G: Dict[str, List[str]]) -> Dict[str, str]:
    """Inverts Employees_G to get group by employee mapping."""
    employees_g: Dict[str, str] = {}
    for group, members in Employees_G.items():
        for employee in members:
            employees_g[employee] = group
    return employees_g

def find_different_desks(assignments: List[Dict[str, str]]) -> List[str]:
    """Find employees who use different desks across days."""
    emp_desks: Dict[str, Set[str]] = {}
    for asg in assignments:
        emp = asg['employee']
        desk = asg['desk']
        emp_desks.setdefault(emp, set()).add(desk)
    return [emp for emp, desks in emp_desks.items() if len(desks) > 1]

def find_lonely_members(
    assignments: List[Dict[str, str]],
    Employees_G: Dict[str, List[str]],
    meeting_days: Dict[str, str]
) -> List[Tuple[str, str, str, str]]:
    """Find employees alone in their zone during group meeting day."""
    lonely: List[Tuple[str, str, str, str]] = []
    for group, day in meeting_days.items():
        zone_members: Dict[str, List[str]] = {}
        group_asgs = [
            a for a in assignments
            if a['day'] == day and a['employee'] in Employees_G.get(group, [])
        ]
        for asg in group_asgs:
            zone = asg['zone']
            zone_members.setdefault(zone, []).append(asg['employee'])
        for zone, members in zone_members.items():
            if len(members) == 1:
                lonely.append((group, members[0], day, zone))
    return lonely

def find_violated_preferences(
    assignments: List[Dict[str, str]],
    Days_E: Dict[str, List[str]]
) -> List[Tuple[str, str, str]]:
    """Find assignments that violate employee day preferences."""
    violated: List[Tuple[str, str, str]] = []
    for asg in assignments:
        emp = asg['employee']
        day = asg['day']
        if emp in Days_E and day not in Days_E[emp]:
            prefs = ', '.join(map(str, Days_E[emp]))
            violated.append((emp, day, prefs))
    return violated

def find_unused_desks(
    assignments: List[Dict[str, str]],
    Desks: List[str],
    Days: List[str]
) -> List[Tuple[str, str, str]]:
    """Find desks that were not used on each day."""
    used = {(asg['desk'], asg['day']) for asg in assignments}
    unused: List[Tuple[str, str, str]] = []
    for desk in Desks:
        for day in Days:
            if (desk, day) not in used:
                unused.append((desk, day, "Este puesto no se utilizó durante toda la semana"))
    return unused

def run_optimization(data: Dict[str, Any], min_attendance: int) -> Dict[str, Any]:
    # 1. Datos de entrada
    Employees   = data['Employees']
    Desks       = data['Desks']
    Days        = data['Days']
    Groups      = data['Groups']
    Zones       = data['Zones']
    Desks_Z     = data['Desks_Z']
    Employees_G = data['Employees_G']
    Days_E      = data['Days_E']
    Desks_E     = data['Desks_E']

    # 2. Precomputaciones
    pref = {(e, t): int(t in Days_E.get(e, [])) for e in Employees for t in Days}
    valid_x = [(e, d, t) for e in Employees for d in Desks_E.get(e, []) for t in Days]
    group_size = {g: len(Employees_G.get(g, [])) for g in Groups}

    # 3. Pesos
    W_extra   = 5
    W_notpref = 8
    W_iso     = len(Employees)
    W_change  = 10

    # 4. Definición del problema
    prob = LpProblem('AsignacionPuestos', LpMaximize)

    # 5. Variables de decisión
    x = {idx: LpVariable(f"x_{idx[0]}_{idx[1]}_{idx[2]}", cat=LpBinary) for idx in valid_x}
    u = {(e, d): LpVariable(f"u_{e}_{d}", cat=LpBinary) for e in Employees for d in Desks_E.get(e, [])}
    m = {(g, t): LpVariable(f"m_{g}_{t}", cat=LpBinary) for g in Groups for t in Days}
    solo = {(g, z, t): LpVariable(f"solo_{g}_{z}_{t}", cat=LpBinary) for g in Groups for z in Zones for t in Days}

    # 6. Función objetivo
    prob += (
        W_extra * lpSum(x.values())
      - W_notpref * lpSum((1 - pref[e, t]) * var for (e, d, t), var in x.items())
      - W_iso   * lpSum(solo.values())
      - W_change * lpSum(u.values())
    )

    # 7. Restricciones
    # R1 & R1b: 1 desk/empleado/día y 1 empleado/desk/día
    for t in Days:
        for e in Employees:
            prob += lpSum(x.get((e, d, t), 0) for d in Desks_E.get(e, [])) <= 1
        for d in Desks:
            prob += lpSum(x.get((e, d, t), 0) for e in Employees) <= 1

    # R2: asistencia mínima semanal
    for e in Employees:
        prob += lpSum(x.get((e, d, t), 0) for (e2, d, t) in valid_x if e2 == e) >= min_attendance

    # R3: un día de reunión por grupo
    for g in Groups:
        prob += lpSum(m[g, t] for t in Days) == 1

    # R4: asistencia obligatoria al día de reunión
    for g in Groups:
        for e in Employees_G.get(g, []):
            for t in Days:
                prob += lpSum(x.get((e, d, t), 0) for d in Desks_E.get(e, [])) >= m[g, t]

    # R6 (Reformulada): aislamiento sin usar n
    for g in Groups:
        M_g = group_size[g]
        for z in Zones:
            for t in Days:
                prob += solo[g, z, t] <= m[g, t]
                prob += lpSum(
                    x.get((e, d, t), 0)
                    for e in Employees_G[g]
                    for d in Desks_Z[z]
                ) <= 1 + M_g * (1 - solo[g, z, t])
                prob += lpSum(
                    x.get((e, d, t), 0)
                    for e in Employees_G[g]
                    for d in Desks_Z[z]
                ) >= solo[g, z, t]

    # R7: relación u-x
    for (e, d, t), var in x.items():
        prob += u[(e, d)] >= var
    for (e, d), uv in u.items():
        prob += lpSum(x.get((e, d, t), 0) for t in Days) <= len(Days) * uv

    # 8. Resolver
    start = time.time()
    prob.solve(PULP_CBC_CMD(msg=False, threads=0, timeLimit=60*60*6, presolve=True))
    print(f"Terminado en {time.time() - start:.2f} s")

    # 9. Extracción de resultados
    assignments = [
        {'employee': e, 'desk': d, 'day': t, 'zone': next(z for z in Zones if d in Desks_Z[z])}
        for (e, d, t), var in x.items() if var.value() == 1
    ]

    # 10. Cálculo de KPIs
    total_asgn = len(assignments)
    total_slots = len(Desks) * len(Days)
    weekly_desk_occupancy = total_asgn / total_slots if total_slots else 0
    total_x = sum(var.value() for var in x.values())
    nonpref_sum = sum((1 - pref[e, t]) * x[(e, d, t)].value() for (e, d, t) in valid_x)
    non_preferred_pct = nonpref_sum / total_x if total_x else 0
    emp_assigned = sum(
        1 for e in Employees
        if any(x.get((e, d, t), 0).value() == 1 for d in Desks_E.get(e, []) for t in Days)
    )
    assigned_pct = emp_assigned / len(Employees) if Employees else 0

    attendees = 0
    alone_count = 0
    together_count = 0
    group_zones_table: List[Dict[str, Any]] = []
    for g in Groups:
        meet = next((t for t in Days if m[g, t].value() == 1), None)
        if meet is None: continue
        zones_count: Dict[str, int] = {}
        for (e, d, t), var in x.items():
            if t == meet and var.value() == 1 and e in Employees_G.get(g, []):
                zsel = next(z for z in Zones if d in Desks_Z[z])
                zones_count[zsel] = zones_count.get(zsel, 0) + 1
        group_zones_table.append({
            'group': g,
            'day': meet,
            'zones': ', '.join(zones_count.keys()),
            'cantidad': ', '.join(str(zones_count[z]) for z in zones_count)
        })
        for cnt in zones_count.values():
            attendees += cnt
            if cnt == 1:
                alone_count += 1
            elif cnt >= 2:
                together_count += cnt

    solo_pct = alone_count / attendees if attendees else 0
    groups_together_pct = together_count / attendees if attendees else 0
    same_desk_count = sum(
        1 for e in Employees
        if sum(u[(e, d)].value() for d in Desks_E.get(e, [])) == 1
    )
    same_desk_pct = same_desk_count / len(Employees) if Employees else 0
    score = value(prob.objective) or 0

    # 11. Métricas adicionales
    employees_g = calculate_employees_by_group(Employees_G)
    different_desks = find_different_desks(assignments)
    meeting_days = {g: next(t for t in Days if m[g, t].value() == 1) for g in Groups}
    lonely_members = find_lonely_members(assignments, Employees_G, meeting_days)
    violated_preferences = find_violated_preferences(assignments, Days_E)
    unused_desks = find_unused_desks(assignments, Desks, Days)

    return {
        'assignments': assignments,
        'group_zones_table': group_zones_table,
        'kpis': {
            'score': round(score, 2),
            'non_preferred_pct': round(non_preferred_pct, 2),
            'assigned_pct': round(assigned_pct, 2),
            'solo_pct': round(solo_pct, 2),
            'groups_together_pct': round(groups_together_pct, 2),
            'weekly_desk_occupancy': round(weekly_desk_occupancy, 2),
            'same_desk_pct': round(same_desk_pct, 2),
        },
        'status': LpStatus[prob.status],
        'employees_g': employees_g,
        'meeting_days': meeting_days,
        'different_desks': different_desks,
        'lonely_members': lonely_members,
        'violated_preferences': violated_preferences,
        'unused_desks': unused_desks
    }


In [6]:
uploader = 'file_input_uploads/instance9.json'

# Modelo de Asignación Semanal de Puestos de Trabajo

## 1. Conjuntos
- **E**: Empleados  
- **D**: Escritorios  
- **T**: Días laborales  
- **G**: Grupos de trabajo  
- **Z**: Zonas físicas del edificio  

## 2. Parámetros
- `pref[e,t] ∈ {0,1}`: 1 si el empleado *e* prefiere asistir el día *t*  
- `Desks_E[e] ⊆ D`: escritorios compatibles con *e*  
- `Desks_Z[z] ⊆ D`: escritorios en la zona *z*  
- `Employees_G[g] ⊆ E`: empleados del grupo *g*  
- `group_size[g]`: tamaño del grupo *g*  
- `min_attendance`: mínimo de días que cada empleado debe asistir  
- `W_extra, W_notpref, W_iso, W_change`: pesos para la función objetivo  

## 3. Variables de Decisión
- $x_{e,d,t} \in \{0,1\}$  
  1 si el empleado *e* usa el escritorio *d* el día *t*

- $u_{e,d} \in \{0,1\}$  
  1 si *e* usa el escritorio *d* al menos una vez en la semana

- $m_{g,t} \in \{0,1\}$  
  1 si el grupo *g* realiza su reunión el día *t*

- $n_{g,z,t} \in \mathbb{Z}_{\ge0}$  
  Número de miembros del grupo *g* en la zona *z* el día *t*

- $\text{solo}_{g,z,t} \in \{0,1\}$  
  1 si solo un miembro del grupo *g* está en la zona *z* el día *t*

## 4. Función Objetivo

Maximizar:
$$
Z =
W_{\text{extra}} \sum_{e \in E} \sum_{d \in Desks_E[e]} \sum_{t \in T} x_{e,d,t}
- W_{\text{notpref}} \sum_{e \in E} \sum_{d \in Desks_E[e]} \sum_{t \in T} (1 - \text{pref}[e,t]) \cdot x_{e,d,t}
- W_{\text{iso}} \sum_{g \in G} \sum_{z \in Z} \sum_{t \in T} \text{solo}_{g,z,t}
- W_{\text{change}} \sum_{e \in E} \sum_{d \in Desks_E[e]} u_{e,d}
$$

## 5. Restricciones

### R1. Un empleado por escritorio y un escritorio por empleado al día
$$
\sum_{d \in Desks_E[e]} x_{e,d,t} \le 1 \quad \forall e \in E,\; t \in T
$$
$$
\sum_{e \in E} x_{e,d,t} \le 1 \quad \forall d \in D,\; t \in T
$$

### R2. Asistencia mínima semanal

Sea **A** el número mínimo de días que cada empleado debe asistir (parámetro `min_attendance`):

$$
\sum_{(d,t) \in D \times T} x_{e,d,t} \ge A \quad \forall e \in E
$$

---

### R6. Aislamiento de empleados en reunión

Sea **M<sub>g</sub>** el tamaño del grupo *g*.

$$
\text{solo}_{g,z,t} \le m_{g,t}
$$

$$
n_{g,z,t} \le 1 + M_g \cdot (1 - \text{solo}_{g,z,t})
$$

$$
n_{g,z,t} \ge \text{solo}_{g,z,t} \quad \forall g \in G,\; z \in Z,\; t \in T
$$


### R3. Un día de reunión por grupo
$$
\sum_{t \in T} m_{g,t} = 1 \quad \forall g \in G
$$

### R4. Asistencia obligatoria al día de reunión del grupo
$$
\sum_{d \in Desks_E[e]} x_{e,d,t} \ge m_{g,t} \quad \forall g \in G,\; e \in Employees_G[g],\; t \in T
$$

### R5. Conteo por zona
$$
n_{g,z,t} = \sum_{e \in Employees_G[g]} \sum_{d \in Desks_Z[z]} x_{e,d,t} \quad \forall g \in G,\; z \in Z,\; t \in T
$$

### R6. Aislamiento de empleados en reunión

Sea $M_g$ el tamaño del grupo $g$.

$$
\text{solo}_{g,z,t} \le m_{g,t}
$$

$$
n_{g,z,t} \le 1 + M_g \cdot (1 - \text{solo}_{g,z,t})
$$

$$
n_{g,z,t} \ge \text{solo}_{g,z,t} \quad \forall g \in G,\; z \in Z,\; t \in T
$$


### R7. Relación entre cambio de escritorio y asignaciones
$$
u_{e,d} \ge x_{e,d,t} \quad \forall (e,d,t)
$$
$$
\sum_{t \in T} x_{e,d,t} \le |T| \cdot u_{e,d} \quad \forall (e,d)
$$


In [7]:
with open(uploader, 'r') as file:
    ins = json.load(file)
data =ins

In [ ]:
# %%
# run optimization usando la función definida previamente
resultados = run_optimization(data, min_attendance=1)

In [9]:
from firebase_admin import firestore
from IPython.display import Markdown, display

firebase_app = setup_firebase_credentials()
if not firebase_app:
    raise RuntimeError('No fue posible inicializar Firebase')

db = firestore.client(firebase_app)
optimizations_col = db.collection('optimizations')

# build a Firestore–safe document from your resultados dict
doc = {
    # copy over everything that is already safe (scalars, maps, arrays of maps)
    **{k: v for k, v in resultados.items()
       if k not in ("lonely_members", "violated_preferences", "unused_desks")},
    # turn each tuple in lonely_members into a map
    "lonely_members": [
        {"group": g, "employee": e, "day": d, "zone": z}
        for (g, e, d, z) in resultados["lonely_members"]
    ],
    # same for violated_preferences
    "violated_preferences": [
        {"employee": emp, "day": day, "preferences": prefs}
        for (emp, day, prefs) in resultados["violated_preferences"]
    ],
    # and unused_desks
    "unused_desks": [
        {"desk": desk, "day": day, "obs": obs}
        for (desk, day, obs) in resultados["unused_desks"]
    ],
}
# now write to Firestore
doc_ref = optimizations_col.document()
doc_ref.set(doc)
idfirebase = doc_ref.id
print('Resultados guardados con ID:', idfirebase)
link = f'https://gestorpuestos.vercel.app/results?id={idfirebase}'
print('🔗 Accede a la vista de resultados:', link)
display(Markdown(f"[🔗 Accede a la vista de resultados](https://gestorpuestos.vercel.app/results?id={idfirebase})"))

Resultados guardados con ID: fjdShLEkShsAcQOjScCX
🔗 Accede a la vista de resultados: https://gestorpuestos.vercel.app/results?id=fjdShLEkShsAcQOjScCX


[🔗 Accede a la vista de resultados](https://gestorpuestos.vercel.app/results?id=fjdShLEkShsAcQOjScCX)

In [10]:
import os

carpeta = 'file_input_uploads'

if os.path.exists(carpeta):
    for archivo in os.listdir(carpeta):
        ruta = os.path.join(carpeta, archivo)
        if os.path.isfile(ruta):
            os.remove(ruta)
    print(f"🧹 Archivos eliminados de la carpeta '{carpeta}'")
else:
    print(f"⚠️ La carpeta '{carpeta}' no existe en este entorno.")


🧹 Archivos eliminados de la carpeta 'file_input_uploads'
